# Drought and Wildfire Situation

As mentioned, record-breaking heat has made headlines around the world this year. It has been warmer then usual on the BC South Coast this spring and early summer; however, the lack of precipitation is even more notable. This section takes a closer look at current [drought](label1) and [wildfire](label2) conditions in the region. More information on drought related to water supply can be found in the next section ([](water_supply.ipynb)).

(label1)=
## Drought

```{figure} img/coq_drawdown.jpg
---
height: 300px
name: drawdown
---
Coquitlam Lake shore in 2022
```

The summer of 2022 was marked by a prolonged seasonal drought. There was almost no precipitation throughout most of the summer, and the fall rains did not appear until October 21. Precipitation during the winter months was near normal, but there were fewer intense atmospheric river events. The heaviest rain came very late in the season on the Easter weekend. 

Conditions dried out and warmed up rapidly at the end of April. Only 111 mm of rain has fallen since April 24. Typically we'd expect around 280 mm of rain during this period. 

```{admonition} **Drought Status** 
:class: attention
The Lower Mainland basin is rated drought level 4 as of **July 27, 2023**
```

```{figure} img/drought_map.png
---
name: drought_map
---
Latest BC drought map from the BC River Forecast Centre [BCRFC](https://governmentofbc.maps.arcgis.com/apps/MapSeries/index.html?appid=838d533d8062411c820eef50b08f7ebc)
```

### Soil Moisture

With very little precipitation during the spring and first half of the summer, it's no surprise that soil moisture levels are exceptionally low. The plot below shows the amount of water (cm^3) in a cubic metre of soil, or the soil water content (SWC). Values are derived from Satellite data and are averaged over the Capilano Watershed area. Soil moisture levels were at record-breaking low levels in late October 2022, and are currently trending even lower this season. 


In [1]:
#Import libraries
import os
import pandas as pd
import subprocess
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import json
from pandas import json_normalize

In [2]:
soil = pd.read_csv('data/wegaw_soil.csv', parse_dates= ['date']) 
soil['year'] = pd.DatetimeIndex(soil['date']).year
soil['month'] = pd.DatetimeIndex(soil['date']).month
soil['DOY'] = pd.DatetimeIndex(soil['date']).dayofyear

In [3]:
title = alt.TitleParams(
   text='Capilano Watershed',
   subtitle="Satellite-derived Soil Moisture",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

soil_line = alt.Chart(soil, title=title).mark_line().encode(
    alt.X('monthdate(date):T', title=None),
    alt.Y('SWC:Q', title='Soil Water Content (cm^3 water/m^3 soil)', scale=alt.Scale(domain=[300000, 450000])),
    alt.Color('year:N', title="Year"),
    tooltip=[alt.Tooltip('date', title="Date"), alt.Tooltip('SWC', title="SWC")]
)

soil_line.properties(width=600)


alt.Chart(...)

In [4]:
df2 = pd.read_csv('data/seyfw_dc.csv', parse_dates= ['datetime']) 
df2['year'] = pd.DatetimeIndex(df2['datetime']).year
df2['month'] = pd.DatetimeIndex(df2['datetime']).month
df2['DOY'] = pd.DatetimeIndex(df2['datetime']).dayofyear

In [5]:
df3 = df2[df2['month'] > 4]
df4 = df3[df3['month'] < 10]

The plot below shows the Drought Code (DC), which is a component of the fire weather index system and an indicator of seasonal drought effects on forest fuels. This value typical climbs throughout the summer months with the highest values seen late in the summer or early in the fall (driest conditions). This season (bold red line) is seeing much higher than average values early in the summer, which is similar to 2015 (violet). Last year is also shown in blue, which saw very high values in mid-to-late October. The dashed black line represents average drought codes, which peak at around 275 at the end of August. 

In [6]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Drought Code - 1988-2023",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

highlight = alt.selection(type='single', on='mouseover',
                          fields=['year'], nearest=True, empty="none")

background = alt.Chart(df4[df4['year'] != 2023], title=title).mark_line(opacity=0.3).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code'),
    color=alt.condition( highlight, 'year:N', alt.value("lightgray"), legend=None),
    tooltip=["year:N", "DC"],
).add_selection(
    highlight
)


year_2023 = alt.Chart(df4[df4['year'] == 2023]).mark_line(color= "red", size= 3).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code')
)

year_2015 = alt.Chart(df4[df4['year'] == 2015]).mark_line(color= "violet", opacity=0.5).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code')
)

year_2022 = alt.Chart(df4[df4['year'] == 2022]).mark_line(color= "darkblue", opacity=0.5).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('DC:Q', title='Drought Code')
)

mean = alt.Chart(df4).mark_line(color='black', opacity=0.5, strokeDash=[4, 2]).encode(
    alt.X('monthdate(datetime):T', title=None),
    alt.Y('mean(DC)', title='Drought Code')
)

year_2023 + year_2015 + year_2022 + mean + background.properties(width=600).interactive()

alt.LayerChart(...)

(label2)=
## Wildfire Conditions

### Provincial Wildfire Summary

Both British Columbia and Canada are experiencing record-breaking wildfire seasons, and the summer is far from over. The season started early in the BC northeast and northern Alberta. Hot and dry conditions have persisted over much of the province, leaving forest fuels tinder dry. As of July 31, there have been 1532 fires this year, and 1.55 million hectares of forest has burned. Nationally, over 12 million hectares of forest has burned, which is a larger area than the country of Portugal. 

Below you can see current BC Wildfire Statistics in comparison to past seasons. All fire zones in the province have seen significant fire activity already this season. Source: [BCWS](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/about-bcws/wildfire-statistics/wildfire-averages#:~:text=Wildfire%20Averages%20%20%20%20Year%20%20,%20375%20%2845%25%29%20%2012%20more%20rows%20).

In [7]:
fires = pd.read_csv('data/fire_stats.csv', parse_dates= ['date'])
fires['year'] = pd.DatetimeIndex(fires['date']).year

In [8]:
#Area burned chart

bar = alt.Chart(fires).mark_bar(opacity=0.8, color='darkgray', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('area_burned:Q', title='Area Burned in Hectares'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('area_burned', title="Area Burned")]
).properties(width=350, height=250, title='BC Area Burned per Year')


bar2 = alt.Chart(fires).mark_bar(opacity=0.8, color='red', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('fire_no:Q', title='Number of Fires'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_no', title="Number of Fires")]
).properties(width=350, height=250, title='BC Number of Fires')


bar | bar2


alt.HConcatChart(...)


Below is a recent provincial fire danger map from the [BCWS](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/prepare/weather-fire-danger/fire-danger)

```{figure} img/fire_dgr.PNG
---
height: 400px
name: fire-danger
---
Fire danger ratings on **July 30, 2023**
```

In [9]:
danger = pd.read_csv('data/seyfw_danger.csv', parse_dates= ['datetime']) 
danger['year'] = pd.DatetimeIndex(danger['datetime']).year
danger['month'] = pd.DatetimeIndex(danger['datetime']).month
danger['DOY'] = pd.DatetimeIndex(danger['datetime']).dayofyear

In [10]:
danger_filter = danger.loc[(danger['DGR']>=4)]

In [11]:
danger_count = danger_filter.groupby('year').count().reset_index()
df_dgr = danger_count.loc[:,'year':'datetime']
df_dgr = df_dgr.rename(columns={"year":"year", "datetime": "fire_danger"})
df_dgr['moving_avg'] = df_dgr['fire_danger'].rolling(10).mean()

### Watershed Wildfire Conditions

```{admonition} **Current Fire Danger:** 
:class: attention
The Watershed fire danger rating is **MODERATE** as of **July 31, 2023**
```

The plot below shows the number of days where fire danger was rated high or extreme in the watershed areas. This varies from season to season, but there has been an increasing trend over time. This year is already above average with two months of summer remaining. 

In [12]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in High or Extreme Fire Danger",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar1 = alt.Chart(df_dgr, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('fire_danger:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_danger', title="Number of Days")]
)

rule1 = alt.Chart(df_dgr).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)

(bar1 + rule1).properties(width=600)

alt.LayerChart(...)

In [13]:
# Import data for crossover conditions and format dates
df = pd.read_csv('data/seyfw_crossover.csv', parse_dates= ['datetime']) 
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['DOY'] = pd.DatetimeIndex(df['datetime']).dayofyear

In [14]:
# Filter dataframe for crossover days
df_filter = df.loc[(df['temp']>=30) & (df['rh'] <= 30)]

In [15]:
# Count crossover days per year
df_count = df_filter.groupby('year').count().reset_index()

In [16]:
# Filter for one column and change the name of the column
df_co = df_count.loc[:,'year':'datetime']
df_co = df_co.rename(columns={"year":"year", "datetime": "crossover_count"})
df_co['moving_avg'] = df_co['crossover_count'].rolling(10).mean()

In [17]:
# Export dataframe
df_co.to_csv('seyfw_crossover_counts.csv', index=False)

The plot below shows the number of days where 30/30 crossover conditions were experienced each year. 30/30 crossover refers to days where temperatures exceed 30 degrees Celsius and relative humidity is lower than 30%. These are very hot and dry days! During these conditions it's likely to see explosive fire behaviour. The red dashed line shows the 10-year moving average. In the past twenty years the number of days with crossover conditions has increase from around 4 to 10. We have seen 9 crossover days so far this season, and should expect to see at least a few more before summer is over. 

In [18]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in Crossover Condition (30/30) per Year",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar = alt.Chart(df_co, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('crossover_count:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2023,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('crossover_count', title="Number of Days")]
)

rule = alt.Chart(df_co).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)

(bar + rule).properties(width=600)

alt.LayerChart(...)

### Wildfire Smoke

British Columbia has been plagued by poor air quality from wildfire smoke most summers since 2015. The Lower Mainland has experienced very smoky conditions in recent years as wildfire smoke from neighbouring regions spreads toward the coast. So far in 2023, smoke from large fires in the BC northeast and central interior has spread east and south. This has resulted in very poor air quality in central and eastern BC, Alberta, and many of the northern US states. Smoky conditions have been felt as far east as New York City. 

```{figure} img/fire_smoke.jpg
---
height: 400px
name: fire-smoke
---
Smoke from BC and AB fires spreading east (July 15, 2023)
```
